In [2]:
import numpy as np
import pandas as pd
from faker import Faker
fake = Faker()

<b>1. Read data from csv.</b>

In [3]:
df = pd.read_csv('igrzyska_list_7.csv')
df['economical_opinion']

0      4.000000
1      3.333333
2      2.333333
3      4.000000
4      3.000000
         ...   
955    3.666667
956    2.666667
957    2.666667
958    2.666667
959    3.333333
Name: economical_opinion, Length: 960, dtype: float64

<b>2. Normalize values in columns q67s1, q67s2, q67s3 and q67s4.</b>

In [4]:
df["q67s1"]=((df["q67s1"]-df["q67s1"].min())
                        /(df["q67s1"].max()-df["q67s1"].min()))

df["q67s2"]=((df["q67s2"]-df["q67s2"].min())
                        /(df["q67s2"].max()-df["q67s2"].min()))

df["q67s3"]=((df["q67s3"]-df["q67s3"].min())
                        /(df["q67s3"].max()-df["q67s3"].min()))

df["q67s4"]=((df["q67s4"]-df["q67s4"].min())
                        /(df["q67s4"].max()-df["q67s4"].min()))

df[["q67s1", "q67s2", "q67s3", "q67s4"]]

,q67s1,q67s2,q67s3,q67s4
0,0.428571,0.428571,0.428571,0.428571
1,0.000000,0.000000,0.000000,0.142857
2,1.000000,1.000000,1.000000,1.000000
3,0.000000,0.000000,0.000000,0.000000
4,0.142857,0.142857,0.142857,0.142857
...,...,...,...,...
955,0.857143,0.857143,0.857143,0.857143
956,0.142857,0.142857,0.142857,0.142857
957,0.000000,0.000000,0.285714,0.428571
958,0.000000,0.000000,0.000000,0.000000


<b>3. Replace NaNs with the most frequent value.</b>

In [5]:
df.loc[df['q13'].isna(), 'q13'] = df['q13'].mode()

print(f"q13 column mode: \t {df['q13'].mode()}",
      f"\nNumber of NaN values: \t {df[df['q13'].isna()]['q13'].count()}")

q13 column mode: 	 0    6.0
dtype: float64 
Number of NaN values: 	 0


<b>4. Create leap_year column based on respondent birth date.</b>

In [6]:
df['year_to_datetime'] = pd.to_datetime(df['URODZONY'], format='%Y')
df['leap_year'] = df['year_to_datetime'].dt.is_leap_year
df[['URODZONY', 'year_to_datetime', 'leap_year']]

,URODZONY,year_to_datetime,leap_year
0,1940,1940-01-01,True
1,1943,1943-01-01,False
2,1936,1936-01-01,True
3,1948,1948-01-01,True
4,1994,1994-01-01,False
...,...,...,...
955,1991,1991-01-01,False
956,1964,1964-01-01,True
957,1975,1975-01-01,False
958,1984,1984-01-01,True


<b>5. Series' Accessors usage example.</b>

In [19]:
full_name_series = pd.Series(data=[fake.name() for i in range(len(df['woj']))])
df['full_name'] = full_name_series
print('=== Day of year ===',
      f"\n{df['year_to_datetime'].dt.dayofweek}",
      '\n\n=== Check if the full_name column ends with \'e\' ===',
      f"\n{df['full_name'].str.endswith('e')}",
      '\n\n=== Get all categories ===',
      f"\n{df['woj'].astype('category').cat.categories}",
      f"\n\n=== Check % of non-empty values in \'q32\' column ===",
      f"\n{df['q32'].astype(pd.SparseDtype(object)).sparse.density}")



=== Day of year === 
0      0
1      4
2      2
3      3
4      5
      ..
955    1
956    2
957    2
958    6
959    5
Name: year_to_datetime, Length: 960, dtype: int64 

=== Check if the full_name column ends with 'e' === 
0      False
1      False
2      False
3      False
4      False
       ...  
955    False
956    False
957    False
958    False
959    False
Name: full_name, Length: 960, dtype: bool 

=== Get all categories === 
Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], dtype='int64') 

=== Check % of non-empty values in 'q32' column === 
0.004166666666666667


<b>6. Create 5 equal birth date groups and a new column assigning respondent to one of these groups.</b>

In [22]:
np.max(df['URODZONY'])

1994